# Agent: People (Phase 1)

This notebook runs a single **People agent** with local random-walk simulation only (no MQTT yet).

In [ ]:
# Cell purpose: import dependencies and load project configuration.
from __future__ import annotations

import random
import time

from simulated_city.config import load_config
from simulated_city.models import Person, PersonState
from simulated_city.movement import apply_boundary_bounce, random_walk_step

config = load_config()
print(f"Loaded base topic from config: {config.mqtt.base_topic}")

In [ ]:
# Cell purpose: read simulation settings from config and initialize 5 people.
simulation_cfg = config.simulation
if simulation_cfg is None:
    raise ValueError("config.yaml must include a 'simulation' section for this notebook")

rng = random.Random(simulation_cfg.seed if simulation_cfg.seed is not None else 42)

PEOPLE_COUNT = simulation_cfg.people_count
STEP_DISTANCE = simulation_cfg.movement.step_distance_m
MAX_TURN_DEG = simulation_cfg.movement.max_turn_deg
TICK_SECONDS = simulation_cfg.movement.tick_s
TOTAL_TICKS = simulation_cfg.movement.total_ticks

# Phase 2 uses config-driven local coordinate bounds.
MIN_X, MAX_X = simulation_cfg.map.min_x, simulation_cfg.map.max_x
MIN_Y, MAX_Y = simulation_cfg.map.min_y, simulation_cfg.map.max_y

names = list(simulation_cfg.names)
colors = list(simulation_cfg.colors)

people: list[Person] = []
for index in range(PEOPLE_COUNT):
    person = Person(
        person_id=f"person_{index + 1}",
        name=names[index % len(names)],
        color=colors[index % len(colors)],
        x=rng.uniform(MIN_X, MAX_X),
        y=rng.uniform(MIN_Y, MAX_Y),
        heading_deg=rng.uniform(0.0, 360.0),
        state=PersonState.RANDOM_WALK,
    )
    people.append(person)

print(f"Initialized {len(people)} people from config-driven settings.")
print(
    f"tick_s={TICK_SECONDS}, total_ticks={TOTAL_TICKS}, "
    f"step_distance={STEP_DISTANCE}, max_turn_deg={MAX_TURN_DEG}"
)

In [ ]:
# Cell purpose: run the local simulation loop and print per-tick state output.
for tick in range(1, TOTAL_TICKS + 1):
    print(f"\nTick {tick}")
    for person in people:
        x1, y1, heading1 = random_walk_step(
            person.x,
            person.y,
            person.heading_deg,
            step_distance=STEP_DISTANCE,
            max_turn_deg=MAX_TURN_DEG,
            rng=rng,
        )
        x2, y2, heading2 = apply_boundary_bounce(
            x1, y1, heading1, MIN_X, MAX_X, MIN_Y, MAX_Y
        )
        person.x = x2
        person.y = y2
        person.heading_deg = heading2

        log = person.to_log_dict()
        print(
            f"- {log['person_id']} ({log['name']}, {log['color']}): "
            f"x={log['x']}, y={log['y']}, heading={log['heading_deg']}, state={log['state']}"
        )

    time.sleep(TICK_SECONDS)

print("\nPhase 1 people-agent simulation complete.")